In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from statistics import mean

# Read in target dataframes
census_df = pd.read_csv("../ETL/ETL-Results/census_data.csv", dtype=str)
cms_df = pd.read_csv("../ETL/ETL-Results/four_mort_measures.csv", dtype=str)



In [2]:
# Remove Some Items due to county issues
cms_df = cms_df[cms_df["State"] != "PR"]
cms_df = cms_df[cms_df["State"] != "GU"]
cms_df = cms_df[cms_df["State"] != "MP"]
cms_df = cms_df[cms_df["State"] != "VI"]

In [3]:
# Remove Some Items due to county issues
cms_df = cms_df[cms_df["County Name"] != "OBRIEN"]
cms_df = cms_df[cms_df["County Name"] != "JEFFRSON DAVIS"]

cms_df = cms_df[~((cms_df["County Name"] == "LASALLE")
                & (cms_df["State"] == "LA"))]
cms_df = cms_df[~((cms_df["County Name"] == "ST. MARYS")
                & (cms_df["State"] == "MD"))]
cms_df = cms_df[~((cms_df["County Name"] == "MCLEAN")
                & (cms_df["State"] == "ND"))]

In [4]:
def fix_county_name(string):

    if string == "DEKALB":
        return "DE KALB"
    if string == "DUPAGE":
        return "DU PAGE"
    elif string == "DISTRICT OF COLUMBIA":
        return "THE DISTRICT"
    elif string == "ANCHORAGE MUNICIPALITY":
        return "ANCHORAGE"
    elif string == "PRINCE GEORGE\'S":
        return "PRINCE GEORGES"
    elif string == "MCDOWELL":
        return "MC DOWELL"
    elif string == "MCHENRY":
        return "MC HENRY"
    elif string == "LASALLE":
        return "LA SALLE"
    elif string == "MCKEAN":
        return "MC KEAN"
    elif string == "MCDUFFIE":
        return "MC DUFFIE"
    elif string == "EAST BATON ROUGE":
        return "E. BATON ROUGE"
    elif string == "LAPORTE":
        return "LA PORTE"
    elif string == "DEWITT":
        return "DE WITT"
    elif string == "JUNEAU CITY AND BOROUGH":
        return "JUNEAU"
    elif string == "KENAI PENINSULA BOROUGH":
        return "KENAI PENINSULA"
    elif string == "MATANUSKA-SUSITNA BOROUGH":
        return "MATANUSKA-SUSITNA"
    elif string == "DESOTO":
        return "DE SOTO"
    elif string == "DOÑA ANA":
        return "DONA ANA"
    elif string == "LA PAZ":
        return "LAPAZ"
    elif string == "MCLEAN":
        return "MC LEAN"
    elif string == "MCCRACKEN":
        return "MC CRACKEN"
    elif string == "MCDONOUGH":
        return "MC DONOUGH"
    elif string == "KETCHIKAN GATEWAY BOROUGH":
        return "KETCHIKAN GATEWAY"
    elif string == "SCOTTS BLUFF":
        return "SCOTT BLUFF"
    elif string == "NORTHUMBERLAND":
        return "NORTHUMBERLND"
    elif string == "MCLENNAN":
        return "MC LENNAN"
    elif string == "MCMINN":
        return "MC MINN"
    elif string == "FAIRBANKS NORTH STAR BOROUGH":
        return "FAIRBANKS NORTH STAR"
    elif string == "MCLEOD":
        return "MC LEOD"
    elif string == "YELLOW MEDICINE":
        return "YELLOW MEDCINE"
    elif string == "SITKA CITY AND BOROUGH":
        return "SITKA BOROUGH"
    else:
        return string

In [5]:
# Save formatted county names
census_df['County Name'] = census_df['County Name'].apply(
    lambda x: fix_county_name(x))

In [6]:
cms_merged_df = pd.merge(
    cms_df, census_df, 
    how="inner", left_on=["State", "County Name"], right_on=["State Abbr", "County Name"]
)

In [7]:
cms_merged_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State_x',
       'ZIP Code', 'County Name', 'Measure ID', 'Measure Name', 'Denominator',
       'Score', 'Lower Estimate', 'Higher Estimate', 'Start Date', 'End Date',
       'State_y', 'State Abbr', 'Household Median Income',
       'Family's Median Income', 'Total Population', 'Percent Poverty',
       'Percent Veteran', 'Percent Married', 'Percent Bachelor',
       'Percent One Race White', 'Percent One Race Black+',
       'Percent One Race American Indian+', 'Percent One Race Asian',
       'Percent One Race Hawaiian+', 'Percent One Race Some Other',
       'Percent Two Race Or More', 'State Code', 'County Code'],
      dtype='object')

In [8]:
cms_merged_df = cms_merged_df.astype({'Percent One Race Black+':'float', 'Percent One Race White': 'float', 'Total Population': 'int'})

In [9]:
black_mean_df = cms_merged_df.groupby(['County Name', 'State Abbr']).agg({'Percent One Race Black+':'mean'})
white_mean_df = cms_merged_df.groupby(['County Name', 'State Abbr']).agg({'Percent One Race White':'mean'})


In [10]:
black_mean = black_mean_df['Percent One Race Black+'].mean()
white_mean = white_mean_df['Percent One Race White'].mean()

In [11]:
black_df = cms_merged_df[cms_merged_df['Percent One Race Black+'] > black_mean]
white_df = cms_merged_df[cms_merged_df['Percent One Race White'] > white_mean]
black_df = black_df[~((black_df['Percent One Race White'] > white_mean) & (black_df['Percent One Race Black+'] > black_mean))]
white_df = white_df[~((white_df['Percent One Race White'] > white_mean) & (white_df['Percent One Race Black+'] > black_mean))]


In [12]:
white_black_df = pd.concat([white_df, black_df])
white_black_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State_x',
       'ZIP Code', 'County Name', 'Measure ID', 'Measure Name', 'Denominator',
       'Score', 'Lower Estimate', 'Higher Estimate', 'Start Date', 'End Date',
       'State_y', 'State Abbr', 'Household Median Income',
       'Family's Median Income', 'Total Population', 'Percent Poverty',
       'Percent Veteran', 'Percent Married', 'Percent Bachelor',
       'Percent One Race White', 'Percent One Race Black+',
       'Percent One Race American Indian+', 'Percent One Race Asian',
       'Percent One Race Hawaiian+', 'Percent One Race Some Other',
       'Percent Two Race Or More', 'State Code', 'County Code'],
      dtype='object')

In [13]:
reduced_df = white_black_df[['Facility Name', 'County Name','City', 'State Abbr', 'Total Population','Measure Name', 
                             'Score','Percent One Race White', 'Percent One Race Black+',]]
reduced_df.head()

,Facility Name,County Name,City,State Abbr,Total Population,Measure Name,Score,Percent One Race White,Percent One Race Black+
298,SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA,MARION,CENTRALIA,IL,37743,Death rate for COPD patients,5.2,92.5,3.7
299,SALEM TOWNSHIP HOSPITAL,MARION,SALEM,IL,37743,Death rate for COPD patients,8.8,92.5,3.7
300,SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA,MARION,CENTRALIA,IL,37743,Death rate for heart failure patients,12.4,92.5,3.7
301,SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA,MARION,CENTRALIA,IL,37743,Death rate for heart attack patients,12.5,92.5,3.7
302,SALEM TOWNSHIP HOSPITAL,MARION,SALEM,IL,37743,Death rate for pneumonia patients,13.5,92.5,3.7


In [14]:
rural_urban_df = reduced_df.groupby(['County Name', 'State Abbr'])
rural_urban_df

In [15]:
# rural_urban = reduced_df.groupby(['County Name', 'State Abbr', 'Measure Name']).agg({
#     'Mortality Percentage' : 'mean',
#     'Percent One Race White' : 'mean',
#     'Percent One Race Black+' : 'mean'
# }).reset_index()

In [16]:
def get_urban_rural(x):
    if x <= 50000: 
        return 'RURAL'
    else: 
        return 'URBAN'

In [17]:
rural_urban_df['Urban_Rural'] = rural_urban_df['Total Population'].apply(get_urban_rural)
rural_urban_df[rural_urban_df['Urban_Rural'] == 'URBAN']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().